# Recurrent Neural Networks and LSTM

**Importing needed libraries**

In [18]:
import numpy as np
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Bidirectional, Embedding

**Recurrent Nerual Network with Keras**

In [5]:
# Initiaing the model
model = keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

# Add an additional hidden layer
model.add(layers.Dense(10))

# View architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          64000     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               24704     
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
Total params: 89,994
Trainable params: 89,994
Non-trainable params: 0
_________________________________________________________________


**LSTM Nerual Network with Keras**

In [6]:

# Example: https://keras.io/guides/working_with_rnns/

# LSTM network example
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          64000     
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


**LSTM NN with on Text Prediction**

-  The novel is the Adventures of Sherlock Holmes by Arthur Conan Doyle

In [9]:
url = "https://raw.githubusercontent.com/bloominstituteoftechnology/data-science-practice-datasets/main/unit_4/sherlock.txt"
response = requests.get(url)
text = response.text

# Strip the \r\n characters
text = text.replace('\r\n', ' ')

**Encoding the Text**

In [10]:
# Encode Data as Chars

# Find the unique characters
chars = list(set(text))

# Lookup tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)}

print('The number of unique characters in the text:', len(chars))

The number of unique characters in the text: 91


**Creating Sequences to train on**

In [13]:
# Create the sequence data
maxlen = 40
step = 5

# Encode the characters using the lookup tables
encoded = [char_int[c] for c in text]

# Initialize empty lists to hold the sequences
sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

# Loop through the entire text
for i in range(0, len(encoded) - maxlen, step): 
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])

print('sequences: ', len(sequences))

sequences:  54974


In [17]:
# Pad sequences so all are equal
seq = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=40)

# Create x and y
# Create arrays of zeros (False)
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

# Turn on the location (set to True) when the character is present
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1

    y[i, next_char[i]] = 1

C:\Users\tyler\AppData\Local\Temp/ipykernel_6736/421023254.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
C:\Users\tyler\AppData\Local\Temp/ipykernel_6736/421023254.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sequences), len(chars)), dtype=np.bool)


In [19]:
model = Sequential()
model.add(Embedding(output_dim=64, input_dim=len(chars)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

**Fit the model**

In [20]:
# Fit the model
model.fit(seq, y, batch_size=32,
          epochs=5, verbose=2)

Epoch 1/5
1718/1718 - 28s - loss: 2.5756 - 28s/epoch - 16ms/step
Epoch 2/5
1718/1718 - 26s - loss: 2.1857 - 26s/epoch - 15ms/step
Epoch 3/5
1718/1718 - 25s - loss: 2.0592 - 25s/epoch - 15ms/step
Epoch 4/5
1718/1718 - 25s - loss: 1.9748 - 25s/epoch - 15ms/step
Epoch 5/5
1718/1718 - 26s - loss: 1.9077 - 26s/epoch - 15ms/step


**Converting outputs back to text**

In [21]:
# Predict and convert text back into characters
def generate_text(model, seed, length):

  encoded = [char_int[c] for c in seed]

  generated = ''
  generated += seed
  model.reset_states()

  start_index = 0 

  for _ in range(length):

      sample = encoded[start_index:start_index+10]      
      sample = np.array(sample)
      sample = np.expand_dims(sample,0)

      pred = model.predict(sample)
      pred = tf.squeeze(pred, 0)
      next_char = np.argmax(pred)
      encoded.append(next_char)
      generated += int_char[next_char]

      start_index += 1

  return generated

In [22]:
# Set the seed text which the model will use to generate the predicted text
seed_text = "I have no data yet it is a capital mistake to theorise before one has data insensibly one begins to twist facts to suit theories"

generate_text(model, seed_text, 400)

'I have no data yet it is a capital mistake to theorise before one has data insensibly one begins to twist facts to suit theoriesmeyeiloattn ts h sasenhiyoynt ye ah the r nedteeore af  tev aoyeyan trt nee af  te eng th then  totke ah terttthe r nd yd dnynn eethetashs   yn n y aoaa  ahe setot  rdn dtn tlhneo k    dwheotot t  tlhntxdetheshe  ssh h rh  thneeeee setoetoooooooosdd er y e aalooaoaoa k  mrr  shetrhtaeaotooethee  rsetllloaerr rhththheeoeien er edtaeeawaweawlhe r   daoetrd ur        eaxeaoaxs ddrntrrr mnaayeatlha  e'

**Increasing Training Epoches**

In [ ]:
# Train with more epochs
model.fit(seq, y, batch_size=32,
          epochs=100, verbose=0)

In [ ]:
seed_text = "I have no data yet it is a capital mistake to theorise before one has data insensibly one begins to twist facts to suit theories"

generate_text(model, seed_text, 400)

'I have no data yet it is a capital mistake to theorise before one has data insensibly one begins to twist facts to suit theoriesdoiinwoktis asty fa-eeiclwegtrgssah bhe   nt.nrlfc-rrtdxoed GevccGsatrtin!y ing prlosa,IoIwoectiiocc.-ihIcpez bhe   cs,.nrrgin?hj—ffcr trmc!séBe  ,eoit-l suent  ew  E_ eTeoaiebmiL4aelay4ve:img” wseuWeoocet  t  t s onn”y”“j”]g i IeenoTTlJ ”   ana”,e”’oeeoIieaepaovP   kt HeCtrt  i xii vO‘zllr1mcsasg?b! ’’ e dn e hh lnhdnnr rs o  h eLcn.   Oa   rtt ddzt eeoIdT   ddc s snnnn”n£sJFsœe,aT e-Meee ioS s e'